<a href="https://colab.research.google.com/github/RiccardoElisei/myRepo/blob/main/IDS%20anomaly%20detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import glob

In [3]:
path = "drive/MyDrive/MachineLearningCVE/"
files = [file for file in glob.glob(path + "**/*.csv", recursive=True)]

In [ ]:
[print(f) for f in files]

In [5]:
dataset = [pd.read_csv(f) for f in files]

In [ ]:
for d in dataset:
    print(d.shape)

In [7]:
dataset = pd.concat([d for d in dataset]).drop_duplicates(keep=False)
dataset.reset_index(drop=True, inplace = True)

In [ ]:
dataset.shape

In [ ]:
dataset.info()

In [ ]:
dataset.describe()

In [ ]:
dataset.head()

In [ ]:
print(dataset[' Label'].unique())
len(dataset[' Label'].unique())

In [ ]:
label_names = dataset[' Label'].unique()

import re

label_names = [re.sub("[^a-zA-Z ]+", "", l) for l in label_names]
label_names = [re.sub("[\s\s]", '_', l) for l in label_names]
label_names = [lab.replace("__", "_") for lab in label_names]

label_names, len(label_names)

In [ ]:
labels = dataset[' Label'].unique()

for i in range(0,len(label_names)):
    dataset[' Label'] = dataset[' Label'].replace({labels[i] : label_names[i]})

dataset[' Label'].unique()

In [ ]:
dataset.isnull().values.any()

In [ ]:
[col for col in dataset if dataset[col].isnull().values.any()]

In [ ]:
dataset['Flow Bytes/s'].isnull().sum()

In [ ]:
before = dataset.shape
dataset.dropna(inplace=True)
after = dataset.shape
before[0] - after[0]

In [ ]:
dataset.isnull().any().any()

In [20]:
import numpy as np

In [21]:
labl = dataset[' Label']
dataset = dataset.loc[:, dataset.columns != ' Label'].astype('float64')

In [ ]:
np.all(np.isfinite(dataset))

In [ ]:
nonfinite = [col for col in dataset if not np.all(np.isfinite(dataset[col]))]
nonfinite

In [ ]:
finite = np.isfinite(dataset['Flow Bytes/s']).sum()

dataset.shape[0] - finite

In [ ]:
finite = np.isfinite(dataset[' Flow Packets/s']).sum()

dataset.shape[0] - finite

In [27]:
dataset = dataset.replace([np.inf, -np.inf], np.nan)

In [28]:
dataset = dataset.merge(labl, how='outer', left_index=True, right_index=True)

In [29]:
dataset.dropna(inplace=True)

In [ ]:
dataset.head()

In [44]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [32]:
np.random.seed(42)
rand_perm = np.random.permutation(dataset.shape[0])

In [34]:
feature_cols = dataset.columns[:-1]

dataset_subset = dataset.iloc[rand_perm[:10000],:]

In [35]:
dataset_subset = dataset_subset.replace([np.inf, -np.inf], np.nan)
dataset_subset.dropna(inplace=True)

In [36]:
data_subset = dataset_subset[feature_cols].values

In [ ]:
pca = PCA(n_components=19)
pca_res = pca.fit_transform(data_subset)

data_subset = None
np.sum(pca.explained_variance_ratio_)

In [ ]:
tsne = TSNE(n_components=2, verbose=0, perplexity=40, n_iter=1000)
tsne_res = tsne.fit_transform(pca_res)
print("done")

In [39]:
dataset_subset['tsne_firstD'] = tsne_res[:,0]
dataset_subset['tsne_secondD'] = tsne_res[:,1]

In [ ]:
plt.figure(figsize=(16,16))

sns.scatterplot(
    x="tsne_firstD", y="tsne_secondD",
    palette=sns.color_palette("hls", n_colors = 15),
    data=dataset_subset,
    hue=' Label',
    legend="full",
    alpha=0.75
)